In [1]:
import os
import sys

sys.path.append(os.path.abspath("../../../"))


In [2]:
import random
from collections import OrderedDict
from copy import deepcopy
from typing import Callable, Dict, List, Optional, Tuple, Iterable

import flwr as fl
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from flwr.common import (EvaluateIns, FitIns, FitRes, MetricsAggregationFn,
                         Parameters, Scalar, Weights, parameters_to_weights,
                         weights_to_parameters)
from flwr.server.client_manager import ClientManager, SimpleClientManager
from flwr.server.client_proxy import ClientProxy
from flwr.server.criterion import Criterion
from flwr.server.strategy import FedAvg
from flwr.server.strategy.aggregate import aggregate, weighted_loss_avg
from prflwr.peer_reviewed.prclient import PeerReviewClient
from prflwr.peer_reviewed.prconfig import PrConfig
from prflwr.peer_reviewed.prserver import PeerReviewServer
from prflwr.peer_reviewed.prstrategy import PeerReviewStrategy
from prflwr.peer_reviewed.strategies.prfedavg import PeerReviewedFedAvg
from prflwr.simulation.app import start_simulation
from prflwr.utils.flwr import import_dataset_utils
from torch.utils.data import DataLoader, Subset, random_split
from torchvision import datasets

import_dataset_utils()


c:\Users\MetBook\Documents\UNIVERSITA\MAGISTRALE\SD21\PROGETTO\FLWR\peer_reviewed_fl\envs\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [107]:
DATASET = "CIFAR10"  # possible values: "CIFAR10" or "CIFAR100"
NUM_CLIENTS = 10
NUM_ROUNDS = 3
LOCAL_EPOCHS = 1
BATCH_SIZE = 128
LR = 0.1
MILESTONES = [60, 120, 160]
LR_DECAY = 0.2
W_DECAY = 5e-04
FRACTION_FIT = 1/3
FRACTION_EVAL = 0
FRACTION_REV = 1/2
SEED = 0
# Set the start method for multiprocessing in case Python version is under 3.8.1
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE = "cpu"
print(f"Training on {DEVICE}")

Training on cpu


In [4]:
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


In [5]:
def load_datasets(
    num_clients: int, dataset: str = "CIFAR10", concentration: float = 1, src: str = "."
):
    if dataset not in ["CIFAR10", "CIFAR100"]:
        raise ValueError(
            "Unknown dataset! Admissible values are: 'CIFAR10' or 'CIFAR100'."
        )
    # Download and transform CIFAR dataset (train and test)
    augmentation = [
        transforms.Pad(4),
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32),
    ]
    transform = [
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
    trainset = getattr(datasets, dataset)(
        os.path.join(src, "./data"),
        train=True,
        download=True,
        transform=transforms.Compose([*augmentation, *transform]),
    )
    testset = getattr(datasets, dataset)(
        os.path.join(src, "./data"),
        train=False,
        download=True,
        transform=transforms.Compose(transform),
    )

    # Split training set into `num_clients` partitions to simulate different local datasets
    targets = np.array(trainset.targets)
    idxs = np.array(range(len(targets)))
    idxs_targets = [idxs, targets]
    train_partitions, _ = fl.dataset.utils.common.create_lda_partitions(
        idxs_targets,
        num_partitions=num_clients,
        concentration=concentration,
        accept_imbalanced=False,
    )

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in map(lambda p: Subset(trainset, p[0]), train_partitions):
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator())
        trainloaders.append(DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=BATCH_SIZE))
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)
    return trainloaders, valloaders, testloader


In [6]:
class Net(nn.Module):
    def __init__(self, num_classes: int) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def train(net, trainloader, epochs: int, lr: float = 0.1):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss().to(DEVICE)
    optimizer = torch.optim.SGD(
        net.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4
    )
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss.item() * labels.size(0)
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net: nn.Module, testloader: DataLoader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss().to(DEVICE)
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item() * labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    print(f"Test loss {loss}, accuracy {accuracy}")
    return loss, accuracy


In [7]:
class FlowerClient(PeerReviewClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net.to(DEVICE)
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def train(self, parameters, config):
        print("training")
        # Read values from config
        current_round = config["current_round"]
        local_epochs = config["local_epochs"]
        lr = config["lr"]
        # Use values provided by the config
        print(f"[Client {self.cid}, round {current_round}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, local_epochs)
        print("trained")
        return get_parameters(self.net), len(self.trainloader), {}

    def review(self, parameters, config):
        print("reviewing")
        loss, num_examples, metrics = self.evaluate(parameters, config)
        metrics[PrConfig.REVIEW_SCORE] = loss
        print("reviewed")
        return [], num_examples, metrics

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


In [44]:
set_seed(SEED)

# Load data
trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS, DATASET, 1 / 3, "../../../")
NUM_CLASSES = len(np.unique(testloader.dataset.targets))

# Create an instance of the model and get the parameters
net = Net(NUM_CLASSES).to(DEVICE)
params = get_parameters(net)
print(sum(p.numel() for p in net.parameters() if p.requires_grad))


Files already downloaded and verified
Files already downloaded and verified
62006


In [101]:
class SelectionCriterion(Criterion):
    def __init__(self, cid: int):
        self.cid = cid

    def select(self, client: ClientProxy) -> bool:
        return True if client.cid != self.cid else False


class FedEs(PeerReviewStrategy):
    def __init__(
        self,
        replay_buffer_maxsize: int = 10,
        fraction_review: float = 0.1,
        fraction_fit: float = 0.1,
        fraction_eval: float = 0.1,
        min_fit_clients: int = 2,
        min_eval_clients: int = 2,
        min_available_clients: int = 2,
        eval_fn: Optional[
            Callable[[Weights], Optional[Tuple[float, Dict[str, Scalar]]]]
        ] = None,
        on_review_config_fn: Optional[Callable[[int], Dict[str, Scalar]]] = None,
        on_fit_config_fn: Optional[Callable[[int], Dict[str, Scalar]]] = None,
        on_evaluate_config_fn: Optional[Callable[[int], Dict[str, Scalar]]] = None,
        accept_failures: bool = True,
        initial_parameters: Optional[Parameters] = None,
        fit_metrics_aggregation_fn: Optional[MetricsAggregationFn] = None,
        evaluate_metrics_aggregation_fn: Optional[MetricsAggregationFn] = None,
    ) -> None:
        self.current_loss: float = None
        self.replay_buffer: List[Dict] = list()
        self.replay_buffer_maxsize = replay_buffer_maxsize
        self.fraction_review = fraction_review
        self.on_review_config_fn = on_review_config_fn
        self.fedavg = FedAvg(
            fraction_fit=fraction_fit,
            fraction_eval=fraction_eval,
            min_fit_clients=min_fit_clients,
            min_eval_clients=min_eval_clients,
            min_available_clients=min_available_clients,
            eval_fn=eval_fn,
            on_fit_config_fn=on_fit_config_fn,
            on_evaluate_config_fn=on_evaluate_config_fn,
            accept_failures=accept_failures,
            initial_parameters=initial_parameters,
            fit_metrics_aggregation_fn=fit_metrics_aggregation_fn,
            evaluate_metrics_aggregation_fn=evaluate_metrics_aggregation_fn,
        )

    @staticmethod
    def copy_extend_dict(source: Dict, extension: Dict):
        new = deepcopy(source)
        for key, val in extension.items():
            new[key] = val
        return new

    def num_review_clients(self, num_available_clients: int):
        """Return the sample size and the required number of available
        clients."""
        num_clients = int(num_available_clients * self.fraction_review)
        return (
            max(num_clients, self.fedavg.min_fit_clients),
            self.fedavg.min_available_clients,
        )

    def add_to_replay_buffer(self, parameters: Parameters):
        if self.replay_buffer_maxsize > len(self.replay_buffer):
            self.replay_buffer.append({"params": parameters})
        else:
            self.replay_buffer.pop(0)
            self.replay_buffer.append({"params": parameters})

    # Standard strategy
    def initialize_parameters(
        self, client_manager: ClientManager
    ) -> Optional[Parameters]:
        return self.fedavg.initialize_parameters(client_manager)

    def configure_evaluate(
        self, rnd: int, parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, EvaluateIns]]:
        return self.fedavg.configure_evaluate(rnd, parameters, client_manager)

    def aggregate_evaluate(
        self,
        rnd: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: BaseException,
    ) -> Tuple[Optional[float], Dict[str, Scalar]]:
        return self.fedavg.aggregate_evaluate(rnd, results, failures)

    def evaluate(
        self, parameters: Parameters
    ) -> Optional[Tuple[float, Dict[str, Scalar]]]:
        loss, metrics = self.fedavg.evaluate(parameters)
        self.current_loss = loss
        return loss, metrics

    # Multiple reviews strategy
    def configure_train(
        self, rnd: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, FitIns]]:
        print("configure_train", flush=True)
        return self.fedavg.configure_fit(rnd, parameters, client_manager)

    def aggregate_train(
        self,
        rnd: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[BaseException],
    ) -> List[Tuple[Optional[Parameters], Dict[str, Scalar]]]:
        print("aggregate_train", flush=True)
        if not results:
            return []
        # Do not aggregate if there are failures and failures are not accepted
        if not self.fedavg.accept_failures and failures:
            return []
        # Convert results
        weights_aggregated = aggregate(
            [
                (parameters_to_weights(fit_res.parameters), fit_res.num_examples)
                for _, fit_res in results
            ]
        )
        print(weights_aggregated[-1])
        return [(weights_to_parameters(weights_aggregated), {})]

    def configure_review(
        self,
        rnd: int,
        review_rnd: int,
        parameters: Parameters,
        client_manager: ClientManager,
        parameters_aggregated: List[Optional[Parameters]],
        metrics_aggregated: List[Dict[str, Scalar]],
    ) -> List[Tuple[ClientProxy, FitIns]]:
        print("configure_review", flush=True)
        # Do not configure federated review if fraction_review is 0
        if self.fraction_review == 0.0:
            return []
        # Parameters and config
        config = {}
        if self.on_review_config_fn is not None:
            # Custom fit config function provided
            config = self.on_review_config_fn(rnd)
        # Prepare review instructions
        config_ins = self.copy_extend_dict(
            config, {"rnd": rnd, "review_rnd": review_rnd, PrConfig.REVIEW_FLAG: True}
        )
        params = parameters_aggregated.pop(0)
        review_ins = FitIns(params, config_ins)
        # Sample clients
        sample_size, min_num_clients = self.num_review_clients(
            client_manager.num_available()
        )
        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )
        # Save parameters in replay buffer
        self.add_to_replay_buffer(parameters_to_weights(params))
        print("replay buffer size:", len(self.replay_buffer))
        # Return client/config pairs
        return [(client, review_ins) for client in clients]

    def aggregate_review(
        self,
        rnd: int,
        review_rnd: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[BaseException],
    ) -> List[Tuple[Optional[Parameters], Dict[str, Scalar]]]:
        print("aggregate_review", flush=True)
        if not results:
            return []
        # Do not aggregate if there are failures and failures are not accepted
        if not self.fedavg.accept_failures and failures:
            return []
        # Aggregate results
        loss_aggregated = weighted_loss_avg(
            [
                (review_res.num_examples, review_res.metrics[PrConfig.REVIEW_SCORE])
                for _, review_res in results
            ]
        )
        self.replay_buffer[-1].setdefault("loss", loss_aggregated)
        return [(None, None)]

    def aggregate_after_review(
        self,
        rnd: int,
        parameters_aggregated: List[Optional[Parameters]],
        metrics_aggregated: List[Dict[str, Scalar]],
        parameters: Optional[Parameters],
    ) -> Optional[Parameters]:
        print("aggregate_after_review", flush=True)
        # Check passed arguments
        assert parameters_aggregated[0] is None and len(parameters_aggregated) == 1
        assert metrics_aggregated[0] is None and len(metrics_aggregated) == 1
        
        # Get weigths and losses
        parameters = parameters_to_weights(parameters)
        print(self.replay_buffer[0]["params"][-1])
        weights_in_buffer = [record["params"] for record in self.replay_buffer]
        losses_in_buffer = [record["loss"] for record in self.replay_buffer]
        print("mean losses:", losses_in_buffer)
        N = len(self.replay_buffer)
        print("replay buffer size:", N)
        
        # Compute lr
        eta = 1 / 2.302585093
        print("lr:", eta)
        alpha = eta / N
        
        # Compute update
        parameters_prime = deepcopy(parameters)
        for i, weights in enumerate(weights_in_buffer):
            for j, tensor in enumerate(weights):
                parameters_prime[j] += alpha * (losses_in_buffer[i] - self.current_loss) * (tensor - parameters[j])
        print(parameters[-1])
        print(parameters_prime[-1])
        
        # Return
        return weights_to_parameters(parameters_prime)

    def stop_review(
        self,
        rnd: int,
        review_rnd: int,
        parameters: Parameters,
        client_manager: ClientManager,
        parameters_aggregated: List[Optional[Parameters]],
        metrics_aggregated: List[Dict[str, Scalar]],
    ) -> bool:
        print("stop_review", flush=True)
        return True


In [102]:
np.concatenate(list(map(lambda t: t.flatten(), params))).shape

(62006,)

In [103]:
from importlib import reload
from prflwr.peer_reviewed import prserver
reload(prserver)
from prflwr.peer_reviewed.prserver import PeerReviewServer
from prflwr.peer_reviewed import prclient
reload(prclient)
from prflwr.peer_reviewed.prclient import PeerReviewClient
from prflwr.peer_reviewed.strategies import prfedavg
reload(prfedavg)
from prflwr.peer_reviewed.strategies.prfedavg import PeerReviewedFedAvg

In [108]:
def client_fn(cid) -> FlowerClient:
    net = Net(NUM_CLASSES).to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)


def evaluate(
    weights: fl.common.Weights,
) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
    net = Net(NUM_CLASSES).to(DEVICE)
    set_parameters(net, weights)  # Update model with the latest parameters
    loss, accuracy = test(net, testloader)
    return loss, {"accuracy": accuracy}


def fit_config(rnd: int):
    lr = LR
    if MILESTONES is not None and LR_DECAY is not None:
        lr *= LR_DECAY ** sum([1 if rnd >= e else 0 for e in MILESTONES])
    config = {
        "current_round": rnd,
        "local_epochs": 1 if rnd < 2 else LOCAL_EPOCHS,
        "lr": lr
    }
    return config


strategy = FedEs(
    fraction_review=FRACTION_REV,
    fraction_fit=FRACTION_FIT,
    fraction_eval=FRACTION_EVAL,
    min_fit_clients=int(FRACTION_FIT * NUM_CLIENTS),
    min_eval_clients=int(FRACTION_EVAL * NUM_CLIENTS),
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.weights_to_parameters(get_parameters(net)),
    on_fit_config_fn=fit_config,
    eval_fn=evaluate,
)
"""strategy = PeerReviewedFedAvg(
    fraction_review=1,
    fraction_fit=1,
    fraction_eval=1,
    min_fit_clients=1,
    min_eval_clients=1,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.weights_to_parameters(get_parameters(net)),
    on_fit_config_fn=fit_config,
    eval_fn=evaluate,
)"""
client_manager = SimpleClientManager()

start_simulation(
    server=PeerReviewServer(client_manager, strategy),
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    num_rounds=NUM_ROUNDS,
    strategy=strategy,
    client_manager=client_manager,
    client_resources={"num_cpus": 1, "num_gpus": 1},
    ray_init_args={"local_mode": True, "include_dashboard": False}
)

INFO flower 2022-06-05 16:02:17,390 | app.py:135 | Ray initialized with resources: {'CPU': 8.0, 'node:127.0.0.1': 1.0, 'object_store_memory': 981633024.0, 'memory': 1963266048.0}
INFO flower 2022-06-05 16:02:17,391 | app.py:151 | Starting Flower simulation running: Config(num_rounds=3, round_timeout=None)
INFO flower 2022-06-05 16:02:17,391 | prserver.py:86 | Initializing global parameters
INFO flower 2022-06-05 16:02:17,393 | server.py:252 | Using initial parameters provided by strategy
INFO flower 2022-06-05 16:02:17,393 | prserver.py:88 | Evaluating initial parameters
INFO flower 2022-06-05 16:02:20,791 | prserver.py:91 | initial parameters (loss, other metrics): 2.3050458293914793, {'accuracy': 0.1011}
INFO flower 2022-06-05 16:02:20,792 | prserver.py:47 | FL starting


Test loss 2.3050458293914793, accuracy 0.1011
configure_train


DEBUG flower 2022-06-05 16:02:20,794 | prserver.py:189 | train_round: strategy sampled 3 clients (out of 10)


:task_name:launch_and_fit
:task_name:launch_and_fit
:task_name:launch_and_fit
training
[Client 3, round 1] fit, config: {'current_round': 1, 'local_epochs': 1, 'lr': 0.1}


:task_name:launch_and_fit
:task_name:launch_and_fit
:task_name:launch_and_fit
DEBUG flower 2022-06-05 16:02:32,696 | prserver.py:202 | train_round received 1 results and 2 failures


...

Epoch 1: train loss 4.924054049558557e+28, accuracy 0.30462222222222224
trained
aggregate_train
[  6.2897      6.777806    3.2472708 -15.386651    5.542212    5.854468
  -1.2435082  -0.7501158 -15.441397    5.017713 ]
configure_review


DEBUG flower 2022-06-05 16:04:17,357 | prserver.py:251 | review_round: strategy sampled 5 clients (out of 10)


replay buffer size: 3


: 

: 